<a href="https://colab.research.google.com/github/hamin32/Extracurricular-activities/blob/main/%EC%9E%90%EB%8F%99%EC%B0%A8_3%EC%B0%A8%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패널 회귀모델 분석(민감도 추정) 단계

In [1]:
# -*- coding: utf-8 -*-
# 안정화 버전: R^2=1 제거, 표본/분산 필터, per-model은 HAC(3)로 추정 (날짜FE 제거)

import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy as pt

IN_PATH  = "/content/hmc_merged_preprocessed.csv"
OUT_TWFE = "/content/hmc_twfe_coeffs.csv"
OUT_PERM = "/content/hmc_model_sensitivities.csv"

# ✅ 독립 변수 목록 (WTI 제거 → 다중공선성 방지)
IV_VARS = ["GSCPI", "krw_per_usd", "Brent_Oil"]
IV_LAG  = [f"{v}_lag1" for v in IV_VARS]

# ----- 하이퍼파라미터 (필요하면 조정) -----
MIN_OBS_PER_MODEL = 50        # 모델별 최소 관측치
MIN_STD_Y         = 1e-4      # 판매증감 분산 최소 기준
R2_MAX_OK         = 0.9999    # 이 이상이면 과적합으로 간주하고 제거

# --- 데이터 로드 ---
df = pd.read_csv(IN_PATH)
if "date" in df.columns:
    try:
        df["date"] = pd.to_datetime(df["date"])
    except Exception:
        pass

need_cols = ["dlog_sales", "Model"] + IV_LAG + ["date"]
missing = [c for c in need_cols if c not in df.columns]
if missing:
    raise ValueError(f"필수 컬럼이 없습니다: {missing}")

dd = df[need_cols].dropna().copy()
dd["date_str"] = dd["date"].dt.strftime("%Y-%m-01")

# --- 안전한 TWFE (참고용: 여전히 불안정할 수 있음) ---
def run_twfe(data: pd.DataFrame) -> pd.DataFrame:
    formula = "dlog_sales ~ " + " + ".join(IV_LAG) + " + C(Model) + C(date_str)"
    y, X = pt.dmatrices(formula, data=data, return_type="dataframe")
    res = sm.OLS(y, X).fit(cov_type="cluster", cov_kwds={"groups": data["Model"]})
    params, bse, pvals, ci = res.params, res.bse, res.pvalues, res.conf_int()
    rows = []
    for var in IV_LAG:
        if var in params.index:
            rows.append({
                "variable": var,
                "beta": params[var],
                "std_err": bse[var],
                "p_value": pvals[var],
                "ci_low": ci.loc[var, 0],
                "ci_high": ci.loc[var, 1],
            })
    out = pd.DataFrame(rows)
    out.insert(0, "spec", "TWFE_cluster_by_Model")
    out["n_obs"] = float(res.nobs)
    out["r2"] = float(res.rsquared)
    return out

# --- per-model 안정화 회귀: 날짜FE 제거 + HAC(3) 표준오차 ---
def run_per_model_safe(data: pd.DataFrame,
                       min_obs: int = MIN_OBS_PER_MODEL,
                       min_std: float = MIN_STD_Y,
                       r2_max_ok: float = R2_MAX_OK) -> pd.DataFrame:
    rows = []
    for m, g in data.groupby("Model"):
        g2 = g[["dlog_sales"] + IV_LAG].dropna().copy()
        if len(g2) < min_obs:
            continue
        if g2["dlog_sales"].std() <= min_std:
            continue
        X = sm.add_constant(g2[IV_LAG])
        y = g2["dlog_sales"]
        try:
            res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags":3})
        except Exception:
            res = sm.OLS(y, X).fit()
        # 과적합/비정상 제거
        r2 = float(res.rsquared) if res.rsquared is not None else np.nan
        if np.isnan(r2) or r2 >= r2_max_ok:
            continue
        for v in IV_LAG:
            if v in res.params.index:
                beta = res.params[v]
                se   = res.bse.get(v, np.nan)
                pval = res.pvalues.get(v, np.nan)
                rows.append({
                    "Model": m,
                    "variable": v.replace("_lag1",""),
                    "beta": float(beta),
                    "std_err": float(se) if pd.notna(se) else np.nan,
                    "p_value": float(pval) if pd.notna(pval) else np.nan,
                    "n_obs": int(res.nobs),
                    "r2": r2,
                })
    out = pd.DataFrame(rows)
    # 남아있는 비정상 값 정리
    out = out.replace([np.inf, -np.inf], np.nan).dropna(subset=["beta", "p_value", "r2"])
    return out

# --- 실행 ---
try:
    twfe = run_twfe(dd)
except Exception as e:
    print("⚠️ TWFE 추정 실패(무시 가능):", e)
    twfe = pd.DataFrame(columns=["spec","variable","beta","std_err","p_value","ci_low","ci_high","n_obs","r2"])

per_model = run_per_model_safe(dd,
                               min_obs=MIN_OBS_PER_MODEL,
                               min_std=MIN_STD_Y,
                               r2_max_ok=R2_MAX_OK)

twfe.to_csv(OUT_TWFE, index=False, encoding="utf-8-sig")
per_model.to_csv(OUT_PERM, index=False, encoding="utf-8-sig")

print("📊 TWFE →", OUT_TWFE, twfe.shape)
print("📈 모델별 민감도(안정화) →", OUT_PERM, per_model.shape)
print("✅ 끝")




/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


📊 TWFE → /content/hmc_twfe_coeffs.csv (3, 9)
📈 모델별 민감도(안정화) → /content/hmc_model_sensitivities.csv (204, 7)
✅ 끝


/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


##결과 요약·해석(상위/하위 민감도, 유의성 필터)

In [2]:
import pandas as pd
import numpy as np

TWFE = "/content/hmc_twfe_coeffs.csv"
PERM = "/content/hmc_model_sensitivities.csv"

twfe = pd.read_csv(TWFE)
perm = pd.read_csv(PERM)

print("=== 전체 평균 민감도 (TWFE) ===")
display(twfe.sort_values("p_value").reset_index(drop=True))

# 모델별 민감도 요약
sig = perm.query("p_value < 0.05").copy()
sig["abs_beta"] = sig["beta"].abs()

print("=== 모델별: 변수별 유의 모델 수 ===")
display(sig.groupby("variable")["Model"].nunique().rename("n_models").reset_index().sort_values("n_models", ascending=False))

print("=== 모델별: 절대 민감도 TOP 20 (p<0.05) ===")
display(sig.sort_values("abs_beta", ascending=False).head(20)[["Model","variable","beta","p_value","r2","n_obs"]])

print("=== 모델별: 변수별 상위 민감 모델 TOP 10 예시 ===")
for var in ["GSCPI","krw_per_usd","WTI_Oil","Brent_Oil"]:
    tmp = sig[sig["variable"]==var].sort_values("abs_beta", ascending=False).head(10)
    print(f"\n## {var} TOP 10")
    display(tmp[["Model","beta","p_value","r2","n_obs"]])


=== 전체 평균 민감도 (TWFE) ===


,spec,variable,beta,std_err,p_value,ci_low,ci_high,n_obs,r2
0,TWFE_cluster_by_Model,GSCPI_lag1,inf,NaN,NaN,NaN,NaN,19707.0,NaN
1,TWFE_cluster_by_Model,krw_per_usd_lag1,-inf,NaN,NaN,NaN,NaN,19707.0,NaN
2,TWFE_cluster_by_Model,Brent_Oil_lag1,inf,NaN,NaN,NaN,NaN,19707.0,NaN


=== 모델별: 변수별 유의 모델 수 ===


,variable,n_models
2,krw_per_usd,15
0,Brent_Oil,10
1,GSCPI,10


=== 모델별: 절대 민감도 TOP 20 (p<0.05) ===


...

,Model,beta,p_value,r2,n_obs
106,Kona,-0.011751,0.004889,0.025101,69
61,GV80,0.009600,0.004379,0.218691,63
52,GV70 (JK EV),-0.009239,0.000230,0.146136,105
112,Kona (HEV),-0.009092,0.003416,0.017429,69
109,Kona (EV),-0.008927,0.031413,0.059210,69
19,Casper (AX),-0.005872,0.023752,0.007351,177
136,Kona N (OS N),0.004773,0.001228,0.154205,105
34,G80 (RG3 EV),-0.004385,0.001417,0.012309,285
16,Avante (CN7),0.002320,0.017033,0.008656,357
25,G70 (IK),0.002308,0.008700,0.005846,213



## WTI_Oil TOP 10


,Model,beta,p_value,r2,n_obs



## Brent_Oil TOP 10


,Model,beta,p_value,r2,n_obs
152,Santa-Fe (MX5 HEV),0.085289,0.016843,0.013431,141
110,Kona (EV),0.052536,0.016882,0.059210,69
164,Santa-Fe (TM PHEV),-0.017190,0.002043,0.109815,144
29,G70 S/B (IK S/B),0.015019,0.029070,0.011004,213
17,Avante (CN7),-0.010632,0.027064,0.008656,357
38,G80 (RG3),-0.009711,0.016901,0.004084,357
11,Avante (CN7 HEV),-0.008545,0.029226,0.009721,321
182,Tucson (NX4 HEV),-0.007443,0.012625,0.004854,324
44,G90 (HI),0.006810,0.015324,0.302900,141
188,Tucson (NX4),-0.005256,0.037883,0.006181,327


##우선배정 리스트 생성 (Priority Calculation)

In [3]:
import pandas as pd
import numpy as np

PERM = "/content/hmc_model_sensitivities.csv"            # per-model 결과(네가 만든 파일)
MERGED = "/content/hmc_merged_preprocessed.csv"          # region_share용
MARGIN = None                                            # 있으면 경로, 없으면 None

# 하이퍼파라미터
ALPHA = 0.05             # p-value 컷
PARTIAL_R2_MIN = 0.02    # 변수 기여 최소(2%)
MIN_OBS = 50
K = 3                    # 설명변수 개수 (GSCPI, krw_per_usd, Brent_Oil)
DF_OFFSET = K + 1        # +상수항

# 1) 로드
perm = pd.read_csv(PERM)
merged = pd.read_csv(MERGED, parse_dates=["date"])

# 2) partial R^2 계산
perm["t_stat"] = perm["beta"] / perm["std_err"]
perm["df"] = (perm["n_obs"] - DF_OFFSET).clip(lower=1)  # 음수/0 방지
perm["partial_r2"] = (perm["t_stat"]**2) / (perm["t_stat"]**2 + perm["df"])

# 3) 필터: 유의 + 기여 + 표본수
filt = (perm["p_value"] <= ALPHA) & (perm["partial_r2"] >= PARTIAL_R2_MIN) & (perm["n_obs"] >= MIN_OBS)
perm_sig = perm.loc[filt].copy()

# 4) region_share 최신값(모델×지역) 결합
last_share = (merged.sort_values(["Model","Region","date"])
              .dropna(subset=["region_share"])
              .groupby(["Model","Region"], as_index=False).tail(1))[["Model","Region","region_share"]]

# (선택) ProfitMargin 결합
if MARGIN:
    margin = pd.read_csv(MARGIN)
    margin.columns = [c.strip() for c in margin.columns]
    cols = [c.lower() for c in margin.columns]
    if set(["model","region","profitmargin"]).issubset(cols):
        # 모델×지역 마진
        def col(name): return margin.columns[[i for i,c in enumerate(cols) if c==name][0]]
        margin = margin.rename(columns={col("model"):"Model", col("region"):"Region", col("profitmargin"):"ProfitMargin"})
        last_share = last_share.merge(margin, on=["Model","Region"], how="left")
    elif set(["model","profitmargin"]).issubset(cols):
        def col(name): return margin.columns[[i for i,c in enumerate(cols) if c==name][0]]
        margin = margin.rename(columns={col("model"):"Model", col("profitmargin"):"ProfitMargin"})
        last_share = last_share.merge(margin, on="Model", how="left")
    else:
        last_share["ProfitMargin"] = 1.0
else:
    last_share["ProfitMargin"] = 1.0

last_share["ProfitMargin"] = pd.to_numeric(last_share["ProfitMargin"], errors="coerce").fillna(1.0)
last_share.loc[last_share["ProfitMargin"]>1, "ProfitMargin"] /= 100.0
last_share["ProfitMargin"] = last_share["ProfitMargin"].clip(0,1)

# 5) 우선배정: |β| × region_share × margin  (유의+기여 통과한 것만)
tmp = perm_sig.merge(last_share, on=["Model"], how="inner")  # 모델 기준으로 지역 확장
tmp["priority"] = tmp["beta"].abs() * tmp["region_share"] * tmp["ProfitMargin"]

# 6) 정렬 및 요약 출력
priority = (tmp.replace([np.inf,-np.inf], np.nan)
              .dropna(subset=["priority"])
              .sort_values("priority", ascending=False)
              [["Model","Region","variable","beta","p_value","partial_r2","n_obs","r2","region_share","ProfitMargin","priority"]])

print("✅ 필터 통과 행:", len(priority))
print(priority.head(30).to_string(index=False))

# 저장(선택)
priority.to_csv("/content/hmc_priority_sig_partialr2.csv", index=False, encoding="utf-8-sig")



✅ 필터 통과 행: 78
             Model         Region  variable      beta  p_value  partial_r2  n_obs       r2  region_share  ProfitMargin  priority
     Kona (SX2 EV) North\nAmerica     GSCPI -0.370100 0.033008    0.032112    141 0.012404      0.649239           1.0  0.240283
              GV80 North\nAmerica     GSCPI -0.338974 0.004338    0.121194     63 0.218691      0.474832           1.0  0.160956
     Kona N (OS N) North\nAmerica     GSCPI  0.211452 0.002165    0.085181    105 0.154205      0.649239           1.0  0.137283
 Santa-Fe (TM HEV) North\nAmerica     GSCPI  0.178576 0.001836    0.031443    303 0.007464      0.649239           1.0  0.115938
          NEXO(FE) North\nAmerica     GSCPI -0.226098 0.003971    0.057103    141 0.007662      0.474832           1.0  0.107359
              GV80         Others     GSCPI -0.338974 0.004338    0.121194     63 0.218691      0.291260           1.0  0.098729
     Kona (SX2 EV)         Others     GSCPI -0.370100 0.033008    0.032112    141 0

##시나리오 분석

##시나리오 설계 원칙

규모 스케일:

zσ 기반(최근 36개월 표준편차) — 충격 일관성↑

pct 기반(평균 대비 ±%) — 환율·유가 같은 레벨 변수에 직관적

기간/경로:

transient(1개월 일시 충격), persistent(지속 충격) 둘 다 제공—영향 비교용
(계산은 동일 delta로 하되, 운영 해석에서 기간만 다르게 본다)

동시충격: 공급망–환율, 공급망–유가 같은 복합 스트레스 포함

타일(quantile): 5/95% 퍼센타일 기반의 극단 시나리오 추가

공분산 기반 방향(PC1): 세 변수의 공동 변동성 주성분 방향으로 스트레스 1세트

##어떻게 해석하나 (요약)

단일 스트레스: GSCPI_up_2σ_persistent → 공급망이 2σ만큼 악화. EV/고부품의존 모델 취약도 확인.

복합 스트레스: Supply_and_FX(공급망+환율↑), Stagflation(공급망↑+유가↑) → 실전형 위기조합.

완화 시나리오: GSCPI_down_1.5σ_relief, Oil_down_20pct → 물량/판촉 재확대 타이밍 탐색.

Tail risk: Tail_risk_high/low → 극단 국면에서 최대손실/최대기회 후보 파악.

Joint_PC1_stress_2σ: 세 변수가 역사적으로 함께 움직인 대표 방향으로 2σ 충격.

| 항목                 | 의미                                                             |
| ------------------ | -------------------------------------------------------------- |
| **Scenario**       | 가정한 외부충격 시나리오 이름 (Tail risk, Stagflation, FX&OIL 등)            |
| **Model**          | 해당 충격에 가장 민감하게 반응하는 차량 모델                                      |
| **Region**         | 분석 대상 지역                                                       |
| **variable**       | 충격의 직접 원인 변수 (예: GSCPI, Brent_Oil, krw_per_usd)                |
| **beta**           | 회귀 민감도 계수 (충격 1단위당 판매율 변화)                                     |
| **shock_strength** | 시나리오 내에서 그 변수가 실제로 얼마나 변했는지 (원단위)                              |
| **region_share**   | 해당 모델이 그 지역에서 차지하는 비중                                          |
| **impact**         | β × shock_strength × region_share → “판매율 영향도” (양수면 수혜, 음수면 타격) |


In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

PERM   = "/content/hmc_model_sensitivities.csv"      # per-model 회귀 결과(필터 전/후 무엇이든 OK)
MERGED = "/content/hmc_merged_preprocessed.csv"      # region_share & 매크로 원천

# 1) 데이터/통계치
perm   = pd.read_csv(PERM)
merged = pd.read_csv(MERGED, parse_dates=["date"])

# 변수 목록
VARS = ["GSCPI","krw_per_usd","Brent_Oil"]

# 최근 36개월 윈도우(데이터가 더 짧으면 전체)
cut_date = merged["date"].max() - pd.DateOffset(months=36)
win = merged[merged["date"] >= cut_date].copy()
if win.empty:
    win = merged.copy()

stats = pd.DataFrame({
    "mean": win[VARS].mean(),
    "std":  win[VARS].std(ddof=1),
    "p05":  win[VARS].quantile(0.05),
    "p95":  win[VARS].quantile(0.95),
    "last": win[VARS].tail(1).iloc[0]
})

def z_to_raw(var, z):
    sd = stats.loc[var,"std"]
    return 0.0 if pd.isna(sd) or sd==0 else float(z*sd)

def pct_to_raw(var, pct):  # 평균 대비 %
    mu = stats.loc[var,"mean"]
    return 0.0 if pd.isna(mu) else float(mu*pct)

def q_shift(var, to_quant):  # 현재수준 → 퍼센타일 수준으로 이동
    target = stats.loc[var,"p95"] if to_quant==0.95 else stats.loc[var,"p05"]
    last   = stats.loc[var,"last"]
    if pd.isna(target) or pd.isna(last):
        return 0.0
    return float(target - last)

# 2) “전문” 시나리오 팩 (원단위 delta로 변환 전의 설계값)
#    type: 'z', 'pct', 'quantile', 'pc1'
SCEN_BLUEPRINT = {
    # 단일 변수 — 표준적 스트레스
    "GSCPI_up_2σ_persistent": {"type":"z", "spec":{"GSCPI": +2.0}},
    "GSCPI_up_1σ_transient":  {"type":"z", "spec":{"GSCPI": +1.0}},
    "GSCPI_down_1.5σ_relief": {"type":"z", "spec":{"GSCPI": -1.5}},

    "FX_up_5pct":  {"type":"pct","spec":{"krw_per_usd": +0.05}},
    "FX_up_10pct": {"type":"pct","spec":{"krw_per_usd": +0.10}},
    "FX_down_5pct":{"type":"pct","spec":{"krw_per_usd": -0.05}},

    "Oil_up_20pct":  {"type":"pct","spec":{"Brent_Oil": +0.20}},
    "Oil_down_20pct":{"type":"pct","spec":{"Brent_Oil": -0.20}},

    # 복합 스트레스 — 운영 의사결정에서 자주 쓰는 조합
    "Supply_and_FX":   {"type":"mixed","spec":{"GSCPI":("z",+2.0), "krw_per_usd":("pct",+0.05)}},
    "Stagflation":     {"type":"mixed","spec":{"GSCPI":("z",+2.0), "Brent_Oil":("pct",+0.20)}},
    "FX_and_Oil":      {"type":"mixed","spec":{"krw_per_usd":("pct",+0.10), "Brent_Oil":("pct",+0.20)}},

    # 극단 타일(Quantile) — 5/95%로 당겨서 충격
    "Tail_risk_high":  {"type":"quantile","spec":{"GSCPI":0.95,"krw_per_usd":0.95,"Brent_Oil":0.95}},
    "Tail_risk_low":   {"type":"quantile","spec":{"GSCPI":0.05,"krw_per_usd":0.05,"Brent_Oil":0.05}},
}

# 3) 공분산 기반 공통방향(PC1) 스트레스 옵션 추가 (있으면 더 전문적)
cov = win[VARS].cov()
try:
    eigvals, eigvecs = np.linalg.eigh(cov.values)
    pc1 = eigvecs[:, -1]  # 최대 고유값 방향
    pc1 = pc1 / np.linalg.norm(pc1)
    # PC1 방향으로 2σ 이동 (변수별 std 반영해 원단위 변환)
    pc1_delta = {var: float(pc1[i]*stats.loc[var,"std"]*2.0) for i,var in enumerate(VARS)}
    SCEN_BLUEPRINT["Joint_PC1_stress_2σ"] = {"type":"abs","spec": pc1_delta}
except Exception:
    pass

# 4) 블루프린트를 원단위(delta)로 변환
def to_raw_shock(name, bp):
    t = bp["type"]
    spec = bp["spec"]
    raw = {v:0.0 for v in VARS}
    if t=="z":
        for v,zv in spec.items():
            raw[v] = z_to_raw(v, zv)
    elif t=="pct":
        for v,p in spec.items():
            raw[v] = pct_to_raw(v, p)
    elif t=="mixed":
        for v,(kind,val) in spec.items():
            raw[v] = z_to_raw(v,val) if kind=="z" else pct_to_raw(v,val)
    elif t=="quantile":
        for v,q in spec.items():
            raw[v] = q_shift(v, q)
    elif t=="abs":
        raw.update(spec)  # 이미 원단위
    else:
        raise ValueError(f"unknown type: {t}")
    return raw

SCENARIOS = {name: to_raw_shock(name, bp) for name,bp in SCEN_BLUEPRINT.items()}

# 5) impact 계산 (per-model β × shock × region_share)
#    region_share: 모델×지역 최신치
perm["Model"] = perm["Model"].astype(str).str.strip()
merged["Model"] = merged["Model"].astype(str).str.strip()

last_share = (merged.sort_values(["Model","Region","date"])
              .dropna(subset=["region_share"])
              .groupby(["Model","Region"], as_index=False).tail(1))[["Model","Region","region_share"]]

# 변수명 정합성
perm["variable"] = perm["variable"].astype(str).str.strip()
perm = perm[perm["variable"].isin(VARS)].copy()

rows = []
for scen, shocks in SCENARIOS.items():
    for _, r in perm.iterrows():
        var = r["variable"]
        beta = r["beta"]
        shock_val = shocks.get(var, 0.0)
        if shock_val == 0 or pd.isna(beta):
            continue
        sub = last_share[last_share["Model"]==r["Model"]]
        if sub.empty:
            continue
        for _, s in sub.iterrows():
            rows.append({
                "Scenario": scen,
                "Model": r["Model"],
                "Region": s["Region"],
                "variable": var,
                "beta": float(beta),
                "shock_strength": float(shock_val),
                "region_share": float(s["region_share"]),
                "impact": float(beta*shock_val*s["region_share"])
            })

impact_df = (pd.DataFrame(rows)
             .replace([np.inf,-np.inf], np.nan)
             .dropna(subset=["impact"]))

# 6) 시나리오 요약 및 TOP 출력
print("=== 시나리오 개요(원단위 delta) ===")
scen_table = pd.DataFrame.from_dict(SCENARIOS, orient="index")[VARS].fillna(0.0)
print(scen_table.round(4).to_string())

print("\n=== 시나리오별 영향 TOP 20 ===")
display(impact_df.sort_values("impact", ascending=False).head(20))

# 저장(선택)
impact_df.to_csv("/content/hmc_scenario_impact_pro.csv", index=False, encoding="utf-8-sig")
print("Saved: /content/hmc_scenario_impact_pro.csv")



=== 시나리오 개요(원단위 delta) ===
                         GSCPI  krw_per_usd  Brent_Oil
GSCPI_up_2σ_persistent  1.6668       0.0000     0.0000
GSCPI_up_1σ_transient   0.8334       0.0000     0.0000
GSCPI_down_1.5σ_relief -1.2501       0.0000     0.0000
FX_up_5pct              0.0000      66.5475     0.0000
FX_up_10pct             0.0000     133.0949     0.0000
FX_down_5pct            0.0000     -66.5475     0.0000
Oil_up_20pct            0.0000       0.0000    16.6738
Oil_down_20pct          0.0000       0.0000   -16.6738
Supply_and_FX           1.6668      66.5475     0.0000
Stagflation             1.6668       0.0000    16.6738
FX_and_Oil              0.0000     133.0949    16.6738
Tail_risk_high          1.6000     -43.6238    19.8629
Tail_risk_low          -1.0400    -164.5761     0.1572
Joint_PC1_stress_2σ    -0.0036     -80.8460    -0.2094

=== 시나리오별 영향 TOP 20 ===


,Scenario,Model,Region,variable,beta,shock_strength,region_share,impact
3313,Tail_risk_high,Santa-Fe (MX5 HEV),North\nAmerica,Brent_Oil,0.085289,19.862881,0.649239,1.099870
3787,Tail_risk_low,Kona,North\nAmerica,krw_per_usd,-0.011751,-164.576141,0.562773,1.088351
3625,Tail_risk_low,GV70 (JK EV),North\nAmerica,krw_per_usd,-0.009239,-164.576141,0.649239,0.987212
1375,Oil_up_20pct,Santa-Fe (MX5 HEV),North\nAmerica,Brent_Oil,0.085289,16.673809,0.649239,0.923281
2752,FX_and_Oil,Santa-Fe (MX5 HEV),North\nAmerica,Brent_Oil,0.085289,16.673809,0.649239,0.923281
2344,Stagflation,Santa-Fe (MX5 HEV),North\nAmerica,Brent_Oil,0.085289,16.673809,0.649239,0.923281
3805,Tail_risk_low,Kona (HEV),North\nAmerica,krw_per_usd,-0.009092,-164.576141,0.562773,0.842061
3796,Tail_risk_low,Kona (EV),North\nAmerica,krw_per_usd,-0.008927,-164.576141,0.562773,0.826830
3526,Tail_risk_low,Casper (AX),North\nAmerica,krw_per_usd,-0.005872,-164.576141,0.649239,0.627396
2569,FX_and_Oil,GV80,North\nAmerica,krw_per_usd,0.009600,133.094912,0.474832,0.606705


Saved: /content/hmc_scenario_impact_pro.csv
